In [1]:
import os
import shutil
import random
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import img_to_array, load_img

def load_images(path, target_shape=(256, 256)):
    images = []
    try:
        for filename in os.listdir(path):
            img_path = os.path.join(path, filename)
            img = Image.open(img_path)
            
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            img = img.resize(target_shape)
            img = np.array(img)
            if img.shape == target_shape + (3,):  
                images.append(img)
            else:
                print(f"Ignoring image {filename} due to incorrect shape: {img.shape}")
    except Exception as e:
        print(f"Error loading images from {path}: {str(e)}")
    return images


source_base_dir = '/kaggle/input/image-pro/Image_Processing'
base_dir = '/kaggle/working/'
original_dir = os.path.join(source_base_dir, 'Original')
pixelated_dir = os.path.join(source_base_dir, 'Pixelated')
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')


os.makedirs(os.path.join(train_dir, 'Original'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'Pixelated'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'Original'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'Pixelated'), exist_ok=True)

def split_data(original_dir, pixelated_dir, train_dir, test_dir, split_ratio):
    original_files = os.listdir(original_dir)
    pixelated_files = os.listdir(pixelated_dir)
    
    paired_files = [(file, file) for file in original_files if file in pixelated_files]

    random.shuffle(paired_files)
    split_index = int(len(paired_files) * split_ratio)
    train_files = paired_files[:split_index]
    test_files = paired_files[split_index:]

    for original_file, pixelated_file in train_files:
        shutil.copy(os.path.join(original_dir, original_file), os.path.join(train_dir, 'Original', original_file))
        shutil.copy(os.path.join(pixelated_dir, pixelated_file), os.path.join(train_dir, 'Pixelated', pixelated_file))
    
    for original_file, pixelated_file in test_files:
        shutil.copy(os.path.join(original_dir, original_file), os.path.join(test_dir, 'Original', original_file))
        shutil.copy(os.path.join(pixelated_dir, pixelated_file), os.path.join(test_dir, 'Pixelated', pixelated_file))


split_ratio = 0.8
split_data(original_dir, pixelated_dir, train_dir, test_dir, split_ratio)


original_images = load_images(os.path.join(train_dir, 'Original'))
pixelated_images = load_images(os.path.join(train_dir, 'Pixelated'))


original_images = np.array(original_images).astype('float32') / 255.0
pixelated_images = np.array(pixelated_images).astype('float32') / 255.0


X_train, X_val, y_train, y_val = train_test_split(pixelated_images, original_images, test_size=0.2, random_state=42)


2024-06-28 20:08:36.015413: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-28 20:08:36.015568: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-28 20:08:36.149589: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def build_deeper_srcnn():
    input_img = Input(shape=(None, None, 3))
    x = Conv2D(128, (9, 9), activation='relu', padding='same')(input_img)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(32, (1, 1), activation='relu', padding='same')(x)
    x = Conv2D(3, (5, 5), activation='sigmoid', padding='same')(x)
    
    model = Model(inputs=input_img, outputs=x)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    
    return model


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)


deeper_model = build_deeper_srcnn()

In [3]:
history = deeper_model.fit(X_train, y_train,
                           epochs=20,
                           batch_size=16,
                           validation_data=(X_val, y_val),
                           callbacks=[reduce_lr])

Epoch 1/20
198/198 ━━━━━━━━━━━━━━━━━━━━ 1860s 9s/step - accuracy: 0.5158 - loss: 0.0734 - val_accuracy: 0.6117 - val_loss: 0.0487 - learning_rate: 0.0010
Epoch 2/20
198/198 ━━━━━━━━━━━━━━━━━━━━ 1865s 9s/step - accuracy: 0.5997 - loss: 0.0502 - val_accuracy: 0.6534 - val_loss: 0.0484 - learning_rate: 0.0010
Epoch 3/20
198/198 ━━━━━━━━━━━━━━━━━━━━ 1892s 9s/step - accuracy: 0.6268 - loss: 0.0485 - val_accuracy: 0.6389 - val_loss: 0.0495 - learning_rate: 0.0010
Epoch 4/20
198/198 ━━━━━━━━━━━━━━━━━━━━ 1881s 10s/step - accuracy: 0.6369 - loss: 0.0484 - val_accuracy: 0.6524 - val_loss: 0.0462 - learning_rate: 0.0010
Epoch 5/20
198/198 ━━━━━━━━━━━━━━━━━━━━ 1878s 9s/step - accuracy: 0.6442 - loss: 0.0478 - val_accuracy: 0.6647 - val_loss: 0.0497 - learning_rate: 0.0010
Epoch 6/20
198/198 ━━━━━━━━━━━━━━━━━━━━ 1882s 10s/step - accuracy: 0.6382 - loss: 0.0484 - val_accuracy: 0.6440 - val_loss: 0.0464 - learning_rate: 0.0010
Epoch 7/20
198/198 ━━━━━━━━━━━━━━━━━━━━ 1885s 10s/step - accuracy: 0.6424 

In [4]:
loss, accuracy = deeper_model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

deeper_model.save('/kaggle/working/deeper_srcnn_model.h5')

25/25 ━━━━━━━━━━━━━━━━━━━━ 117s 5s/step - accuracy: 0.6665 - loss: 0.0446
Validation Loss: 0.04453545808792114, Validation Accuracy: 0.6676664352416992
